# Base & Encapsulation

### DictLike

In [ ]:
# Create a class which builds dict based on two arrays: keys and values
# Hints: class definition, __init__, zip([1,2,3], [3,4,5]) => [(1,3), (2,4), (3,5)]

class Container:
    
    def __init__(self, values, keys):
        self.items = dict(list(zip(keys, values)))

# Example:
c = Container(values=[1,2,3], keys=["a", "b", "c"])
print(c.items)
assert c.items == {"a": 1, "b": 2, "c": 3}

In [ ]:
# Also, see how to work with 'namedtuple'

from collections import namedtuple

Storage = namedtuple("Storage", ["values", "keys"])

# # the namedtuple above is the equivalent of the following code:
# class Storage:
#     def __init__(self, values, keys):
#         self.values = values
#         self.keys = keys

# s = Storage(keys=[1,2,3], values=[3,4,5])
s = Storage([1,2,3], [3,4,5])
s.values

### Properties/Setters

In [21]:
# example of simple property "a".
# In setter-method you can see additional checking of
# input value type and raising our custom exception (SomeError)
# in case of not int value.

class SomeError(Exception):
    pass


class A:
    def __init__(self):
        self._a = 1
        
    @property
    def a(self):        
        return self._a
    
    @a.setter
    def a(self, value):
        if not isinstance(value, int):
            raise SomeError("not int")
        self._a = value

a_obj = A()

# Try to use here (instead of "df") some int value (123)
# and run this cell again. You will see no error
a_obj.a = "df"

SomeError: not int

### DBData

In [ ]:
# 1
# Create a class which contains data for db access.
# Can accept url+user/pass OR all attributes explicitly.
# Raise DBException (define it) in case of wrong type:
# - int(convert 'port' to int)/str,
# - username/password/etc with spaces
# 2
# Make all args as properties (properties + setters)
# and all validation should be performed in setters

# Hists: "s" in "sdfdfg" => True, "1231".isdigit() => True,
# "12f31".isdigit() => False,
# 'None' for default args and then check:
# if some_var is None: or if some_var is not None


class DBData:
    
    def __init__(self, user, password, host=None,
                 db=None, port=None, url=None):
        self.user = user
        self.password = password
        if url is not None:
            items = url.split('/')
            self.db = items[-1]
            
            host_and_port = url.split('/')[2].split(':')
            self.host = host_and_port[0]
            self._port = host_and_port[1]
        else:
            self.db = db
            self._port = port
            self.host = host
    
    @property
    def port(self):
        return self._port
    
    @port.setter
    def port(self, value):
        # some check (or any other code) can be placed here
        self._port = value

# The example below shows how to get values from string like that:

# In [7]: url="mysql://localhost:3306/test"
# In [8]: url.split('/')
# Out[8]: ['mysql:', '', 'localhost:3306', 'test']

# In [9]: url.split('/')[2].split(':')
# Out[9]: ['localhost', '3306']

# Example:
connector = DBData(user="nice_admin", password="123",
                   host="localhost", db="test", port=3306)

connector_short = DBData(url="mysql://localhost:3306/test",
                         user="nice_admin", password="123")

# You will se the same values in both cases
print(connector.password, connector.user,
      connector.host, connector.db, connector.port)
print(connector_short.password, connector_short.user,
      connector_short.host, connector_short.db, connector_short.port)

In [16]:
class DBException(Exception):
    pass


class DBData:
    
    def __init__(self, user, password, host=None,
                 db=None, port=None, url=None):
        self.user = user
        self.password = password
        if url is not None:
            items = url.split('/')
            self.db = items[-1]
            
            host_and_port = url.split('/')[2].split(':')
            self.host = host_and_port[0]
            self.port = host_and_port[1]
        else:
            self.db = db
            self.port = port
            self.host = host
            
        if " " in user:
            raise DBException(f"user has spaces: {user}")
        elif " " in password:
            raise DBException("password has spaces")
        elif any([not isinstance(v, str) for v in [user, password,
                                                   self.host,
                                                   self.db]]):
            raise DBException(f"some arg is not str: host={self.host};"
                              f"db={self.db};user={user}")
        elif (isinstance(self.port, str) and not self.port.isdigit()) or \
                not (isinstance(self.port, str) or isinstance(self.port, int)):            
            raise DBException(f"port {self.port} is not int")

# Try to change args and see different errors
DBData(user="nice_admin", password="123",
       host="localhost", db="test", port="123")

In [19]:
class DBException(Exception):
    pass


class DBData:
    
    def __init__(self, user, password, host=None,
                 db=None, port=None, url=None):
        self.user = user
        self._password = password
        if url is not None:
            items = url.split('/')
            self.db = items[-1]
            
            host_and_port = url.split('/')[2].split(':')
            self.host = host_and_port[0]
            self._port = int(host_and_port[1])
        else:
            self.db = db
            self._port = port
            self.host = host
            
        if " " in user:
            raise DBException("user has spaces")
        elif " " in password:
            raise DBException("password has spaces")
        elif any([not isinstance(v, str) for v in [user, password,
                                                   self.host,
                                                   self.db]]):
            raise DBException("some arg is not str")
        elif (isinstance(self.port, str) and not self.port.isdigit()) or \
                not (isinstance(self.port, str) or isinstance(self.port, int)): 
            raise DBException("port is not int")

    @property
    def port(self):
        return self._port
    
    @port.setter
    def port(self, value):
        if (isinstance(value, str) and
                not value.isdigit()) or \
                    not isinstance(value, int):
            raise DBException("port is not int")
        self._port = value
        
    @property
    def password(self):
        return self._password
    
    @password.setter
    def password(self, value):
        if not isinstance(value, str) or \
                " " in value:
            raise DBException("password is not str or contains spaces")
        self._password = value
        

d = DBData(user="nice_admin", password="123",
           host="localhost", db="test", port=3306)

# # Try this and get error
# d.port = "somestr"
# d.password = 123
d.password = "123"

In [20]:
import unittest
 

class TestDBData(unittest.TestCase):
 
    def test_attrs_all(self):
        connector = DBData(user="nice_admin", password="123", host="localhost", db="test", port=3306)
        self._run_attr_asserts(connector)
        
    def test_attrs_url(self):
        connector = DBData(url="mysql://localhost:3306/test", user="nice_admin", password="123")
        self._run_attr_asserts(connector)
 
    def _run_attr_asserts(self, connector):
        self.assertEqual("nice_admin", connector.user)
        self.assertEqual("123", connector.password)
        self.assertEqual("localhost", connector.host)
        self.assertEqual("test", connector.db)
        self.assertEqual(3306, connector.port)
        
    def test_user_space(self):
        self.assertRaises(DBException, DBData, user="nice admin",
                          password="123 ", host="localhost",
                          db="test", port=1)

    def test_pass_space(self):
        pass
    
    def test_port_not_int(self):
        pass
    
    def test_host_empty(self):
        pass
    
    def test_db_empty(self):
        pass
    
# just specific args in order to run in jupyter
unittest.main(argv=['first-arg-is-ignored'], exit=False)

# in case of a separate file, just use
# unittest.main()

.......
----------------------------------------------------------------------
Ran 7 tests in 0.015s

OK


### Save as JSON

In [2]:
# JSON processing: https://docs.python.org/3/library/json.html
import json

json.dump?

In [ ]:
import json

data = {"a": 1}
with open('data.json', 'w') as f:
    json.dump(data, f)

In [3]:
# Process only first level of input dicts
# Use json.dumps, write to file

# In [1]: a = {"a": 1}
# In [2]: b = {"b": 2}
# In [4]: a.update(b)
# In [5]: a
# Out[5]: {'a': 1, 'b': 2}

class JSONSave:
    
    def __init__(self, init_dict):
        self.data = init_dict
    
    def add(self, new_dict):
        self.data.update(new_dict)
    
    def save(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.data, f)

# Example:
some_value = {"some_inital_key": 123.1, "another_initial": [1,2,3]}

jss = JSONSave(init_dict=some_value)
jss.add({"some_key": [1,2,3], "another": ""})
jss.add({"some_another_key": "123"})
jss.save(filename="result.json")

### Adapter

Adapters are all about altering the interface. Like using a cow when the system is expecting a duck

In [5]:
# Wrapper for list with different methods:
# - append -> add
# - index -> indexof
# - remove -> remove_by_ind
# - pop -> remove_last
# - __str__

class WrappedList:
    
    def __init__(self, init_lst):
        self.lst = init_lst
    
    def add(self, item):
        self.lst.append(item)
        
    def indexof(self, item):
        return self.lst.index(item)
    
    def remove_by_ind(self, index):
        return self.lst.pop(index)
    
    def remove_last(self):
        return self.lst.pop()
    
    def __str__(self):
        return str(self.lst)

# Example:
wlst = WrappedList([1, 2, 3])
wlst.add(12)  # [1, 2, 3, 12]

In [7]:
# Tests
import unittest
 

class TestWrappedList(unittest.TestCase):
 
    def setUp(self):
        """
        A special method that
        runs before every test_* method
        """
        self.wlst = WrappedList([345, 3354, -100500])
        
    def test_add(self):
        prev_wlst = self.wlst.lst[:]
        new_item = 4

        self.wlst.add(new_item)
        self.assertEqual(prev_wlst + [new_item], self.wlst.lst)
        
    def test_indexof(self):
        self.assertEqual(2, self.wlst.indexof(-100500))
    
    def test_removebyind(self):
        prev_wlst = self.wlst.lst[:]
        remove_index = 1
        
        removed_item = self.wlst.remove_by_ind(remove_index)
        
        self.assertEqual(prev_wlst[remove_index], removed_item)
        self.assertEqual(prev_wlst[0:remove_index] +
                         prev_wlst[remove_index + 1:],
                         self.wlst.lst)
    
    def test_removelast(self):
        prev_wlst = self.wlst.lst[:]    
        removed_item = self.wlst.remove_last()
    
        self.assertEqual(prev_wlst[:-1], self.wlst.lst)
        self.assertEqual(prev_wlst[-1], removed_item)
    
    def test_str(self):
        arr = self.wlst.lst
        self.assertEqual(str(arr), str(self.wlst))
        
        
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.009s

OK


# Inheritance & Polymorphism

### Observer

The observer pattern is a software design pattern in which an object, called the subject, maintains a list of its dependents, called observers, and notifies them automatically of any state changes, usually by calling one of their methods.

In [18]:
# Implement the following Observer Pattern

class Subject:

    def __init__(self):
        self._observers = []

    def attach(self, observer):
        self._observers.append(observer)

    def detach(self, observer):
        self._observers.remove(observer)

    def notify(self):
        for observer in self._observers:
            observer.update(self)


class Data(Subject):

    def __init__(self, name):
        super().__init__()
        self._data = None
        self.name = name

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, value):
        self._data = value
        self.notify()


class HexViewer:

    def update(self, subject):
        print('HexViewer: Subject %s has data 0x%x' %
              (subject.name, subject.data))


class DecimalViewer:

    def update(self, subject):
        print('DecimalViewer: Subject %s has data %d' %
              (subject.name, subject.data))

In [19]:
data1 = Data('Data 1')
data2 = Data('Data 2')

view1 = DecimalViewer()
view2 = HexViewer()

data1.attach(view1)
data1.attach(view2)

data2.attach(view2)
data2.attach(view1)

# See how Viewers react to this
print("Setting Data 1 = 10")
data1.data = 10

print("\nSetting Data 2 = 15")
data2.data = 15
print("\nSetting Data 1 = 3")
data1.data = 3
print("\nSetting Data 2 = 5")
data2.data = 5

# And now without any reaction
print("\nDetach HexViewer from data1 and data2.")
data1.detach(view2)
data2.detach(view2)
print("\nSetting Data 1 = 10")
data1.data = 10
print("\nSetting Data 2 = 15")
data2.data = 15

Setting Data 1 = 10
DecimalViewer: Subject Data 1 has data 10
HexViewer: Subject Data 1 has data 0xa

Setting Data 2 = 15
HexViewer: Subject Data 2 has data 0xf
DecimalViewer: Subject Data 2 has data 15

Setting Data 1 = 3
DecimalViewer: Subject Data 1 has data 3
HexViewer: Subject Data 1 has data 0x3

Setting Data 2 = 5
HexViewer: Subject Data 2 has data 0x5
DecimalViewer: Subject Data 2 has data 5

Detach HexViewer from data1 and data2.

Setting Data 1 = 10
DecimalViewer: Subject Data 1 has data 10

Setting Data 2 = 15
DecimalViewer: Subject Data 2 has data 15


### Chain of responsibility

This pattern gives us a way to treat a request using different methods, each one addressing a specific part of the request.

In [ ]:
class FilterApplier:

    def __init__(self, filters=None):
        self._filters = []
        if filters is not None:
            self._filters += filters

    def filter(self, content):
        for fil_obj in self._filters:
            content = fil_obj.filter(content)
        return content

In [ ]:
# Let's implement some filters for text content:
# 1
# Create a base class "BaseFilter" with method "filter" that takes
# some string and return filtered string.
# 2
# Then, inherit it with three filters:
# - AdsFilter: filter words "not a spam"/"you win"/"winner"
# - PoliticsFilter: politics/tramp/russia/war
# - CatFilter: cat/kitty/kitten
# 3
# Write unittests: provide a separate test for each filter
# and one common test for FilterApplier
...

cat_filter = CatFilter(some_init_args)
ads_filter = AdsFilter(some_init_args)
politics_filter = PoliticsFilter(some_args)

filter_obj = FilterApplier([
                cat_filter,
                ads_filter,
                politics_filter])
filtered_content = filter_obj.filter(content)

### Figures Drawing

In [ ]:
# 1
# Base class with symbol which will be used for drawing
# 2
# Implement Circle/Rectangular/SquareTriangular with method draw()
# 3
# Implement scene with methods: append/pop/show

# Hints: use just print() with appropriate formatting

### Context Manager

In [ ]:
# Context Manager, which search file by pattern and read all matched files
    
# Example:    
# dir/
#    some_file.txt
#    doc.pdf
#    imp.txt
#    another_file.txt


with SmartContextManager("file") as f:
    print(f.read())

# Result: print data from both files (some_file.txt and another_file.txt)


class SmartContextManager:
    
    def __init__(self, filepart):
        ...
    
    def __enter__(self):
        ...

    def __exit__(self, *args):
        ...

### Loggers

In [ ]:
# Implement 3 loggers: Base Class and 3 Children: to file (specified),
# to file (default in tmp dir), to console (print)
# interface: info/debug/error/formatter(as a property)

import logging
import logging.handlers

logger = logging.getLogger("custom")
logger.setLevel(logging.DEBUG)

handler_file_def = logging.handlers.RotatingFileHandler
handler_console = logging.StreamHandler
formatter = logging.Formatter

handler_console.setFormatter
logger.addHandler

logger.debug('This message should go to the log file')

### Money class

In [ ]:
# Implement Money class:
#    - store two variables: int + coins
#    - implement +, -, ==, !=, correct precision, unitttest (with classes)

# Hints: __add__, __sub__, __eq__, __ne__

### Bank Account

In [ ]:
# Bank Account, which contains
#    - Person/Address
#    - ContactInfo object, which inherits from Person/Address, and contains unique id (see uuid)
#    - Money object
# implement:
#    - str/repr
#    - recharge/withdraw
#    - implement logging for each operation (logging object as a arg in init)